In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import seaborn as sns

In [ ]:
sns.set_style('whitegrid')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import xgboost

In [ ]:
np.random.seed(1767)

Your task is to predict the Site EUI for each row, given the characteristics of the building and the weather data for the location of the building. 

In [ ]:
always_drop_columns = ['days_with_fog', 'direction_peak_wind_speed', 'max_wind_speed', 'direction_max_wind_speed']

In [ ]:
df = pd.read_csv('./data/train.csv.zip').drop(columns=['id'] + always_drop_columns)
df.columns = df.columns.str.lower()

In [ ]:
test_df = pd.read_csv('C:/Users/k202141/Downloads/test.csv.zip').drop(columns=always_drop_columns)
test_df.columns = test_df.columns.str.lower()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
cat_columns = list(df.select_dtypes(['object']).columns)
cat_columns

In [ ]:
for cc in cat_columns:
    df[f'{cc}_'] = df[cc].astype('category').cat.codes
    test_df[f'{cc}_'] = test_df[cc].astype('category').cat.codes    
df = df.drop(columns=cat_columns)

In [ ]:
df.info()

In [ ]:
df.corr()['site_eui'].sort_values().head()
df.corr()['site_eui'].sort_values().tail()

In [ ]:
df.corr()['energy_star_rating'].sort_values().tail()

## Random Forest

In [ ]:
features = list(df.drop(columns=['site_eui', 'year_factor']).columns)
transform_labels = True
transform_features = True
fill_missing_values = False
use_clipping = True

In [ ]:
X = df.loc[:, features]
y = df.loc[:, 'site_eui']
year_factor = df.loc[:, 'year_factor']
X.shape

X

In [ ]:
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneGroupOut

#X_train = X[year_factor < 6]
#y_train = y[year_factor < 6]
#X_valid = X[year_factor == 6]
#y_valid = y[year_factor == 6]

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=200)

logo = LeaveOneGroupOut()
print(logo.get_n_splits(X, y, year_factor))

if fill_missing_values:
    fill_values = X.median()
    X = X.fillna(fill_values)
    
if use_clipping:
    clip_lower = X.mean() - X.std() * 3
    clip_upper = X.mean() + X.std() * 3

    X = X.clip(lower=clip_lower, upper=clip_upper, axis=1)    

In [ ]:
if transform_features:
    scaler = MinMaxScaler() #StandardScaler()
    X = scaler.fit_transform(X)

if transform_labels:
    qtrafo = QuantileTransformer(output_distribution='normal')
    y = qtrafo.fit_transform(y.values.reshape(-1, 1)).squeeze()

In [ ]:
fig, ax = plt.subplots(1, 6, sharex=True, sharey=True, figsize=(24, 4))
ax = ax.flatten()
i = 0

models = []
feature_importance_df = []

for train_index, valid_index in logo.split(X, y, year_factor):
    X_train = X[train_index]
    y_train = y[train_index]
    X_valid = X[valid_index]
    y_valid = y[valid_index]

    rfr = xgboost.XGBRegressor(n_estimators=1000)
    rfr = rfr.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
                  early_stopping_rounds=10, 
                  verbose=0)
    
    print('*'*40)
    print('Evaluating on year_factor', year_factor.iloc[valid_index].unique())
    print('R2:   ', rfr.score(X_valid, y_valid))
    
    y_valid_hat = rfr.predict(X_valid)
    if transform_labels:
        y_valid = qtrafo.inverse_transform(y_valid.reshape(-1, 1)).squeeze()
        y_valid_hat = qtrafo.inverse_transform(y_valid_hat.reshape(-1, 1)).squeeze() 
        
    print('RMSE: ', np.mean((y_valid - y_valid_hat)**2))
    
    ax[i].hexbin(y_valid, y_valid_hat, cmap='viridis', norm=colors.LogNorm(vmin=1, vmax=100), mincnt=1)
    ax[i].set_aspect('equal')
    ax[i].plot(np.arange(0, 1000), np.arange(0, 1000), 'r--')
    i += 1
    
    models.append(rfr)
    feature_importance_df.append(pd.DataFrame(rfr.feature_importances_, index=features))
        
feature_importance_df = pd.concat(feature_importance_df, axis=1)

In [ ]:
feature_importance_df.mean(axis=1).sort_values(ascending=False)

### Generate test set predictions

In [ ]:
X_test = test_df.loc[:, features]

if fill_missing_values:
    X_test = X_test.fillna(fill_values)
    
if use_clipping:
    X_test = X_test.clip(lower=clip_lower, upper=clip_upper, axis=1)

print(X_test.shape, len(features))

if transform_features:
    X_test = scaler.transform(X_test)

print(pd.DataFrame(X_test).describe())

In [ ]:
y_hat = []

for rfr in models:
    yy = rfr.predict(X_test)
    if transform_labels:
        yy = qtrafo.inverse_transform(yy.reshape(-1, 1)).squeeze()
    y_hat.append(yy)
    
y_hat = np.mean(np.asarray(y_hat), axis=0)

In [ ]:
test_df['site_eui'] = y_hat
submission_df = test_df.loc[:, ['id', 'site_eui']]

In [ ]:
submission_df['site_eui'].hist()

In [ ]:
submission_file = '20220125_random_forest_07.csv'

if os.path.exists(os.path.join('submissions', submission_file)):
    assert False
submission_df.to_csv(os.path.join('submissions', submission_file), index=False)